In [ ]:
#!/usr/bin/env python
"""N‑PX Data‑Quality Notebook (full)
===================================
Run after the ETL has finished. Fails fast (sys.exit != 0) on any
integrity problem.  Covers:
  • Row counts per table
  • FK orphan checks (both directions)
  • NOT‑NULL sanity
  • Business‑logic checks (dup keys, negative shares, date sanity, etc.)
  • Parse‑success rate and sample join preview
Set PGHOST / PGUSER / PGPASSWORD / PGDATABASE as env‑vars before running.
"""

import os, sys, psycopg2, pandas as pd
from psycopg2.extras import RealDictCursor

# ── helpers ───────────────────────────────────────────────────────────┐

def conn():
    return psycopg2.connect(
        host=os.environ['PGHOST'],
        user=os.environ.get('PGUSER'),
        password=os.environ.get('PGPASSWORD'),
        dbname=os.environ.get('PGDATABASE'),
        cursor_factory=RealDictCursor,
    )

def q(sql, *args):
    with conn() as c, c.cursor() as cur:
        cur.execute(sql, args)
        return cur.fetchall()

def qc(label, sql, *args, expect_zero=False):
    cnt = q(sql, *args)[0]['cnt']
    status = 'OK' if (cnt==0 if expect_zero else True) else 'FAIL'
    print(f"{label:<45} ➜  {cnt:>6}   {status}")
    if expect_zero and cnt!=0:
        print('‼️  check failed, aborting'); sys.exit(1)

# ── 1. row counts ─────────────────────────────────────────────────────┘
print('\nROW COUNTS')
for tbl in [
  'form_npx','institutional_manager','series','series_class','other_reporting_person',
  'proxy_voting_record','matter_category','proxy_voting_record_category',
  'voting_record_manager','voting_record_series']:
    qc(tbl, f'SELECT COUNT(*) cnt FROM {tbl}')

# ── 2. FK orphans ─────────────────────────────────────────────────────
print('\nORPHAN CHECKS (expect 0)')
checks = {
    'manager→form' : "SELECT COUNT(*) cnt FROM institutional_manager m LEFT JOIN form_npx f USING(form_id) WHERE f.form_id IS NULL",
    'series→form'  : "SELECT COUNT(*) cnt FROM series s LEFT JOIN form_npx f USING(form_id) WHERE f.form_id IS NULL",
    'series_class→series': "SELECT COUNT(*) cnt FROM series_class sc LEFT JOIN series s USING(series_id) WHERE s.series_id IS NULL",
    'vote→form'    : "SELECT COUNT(*) cnt FROM proxy_voting_record v LEFT JOIN form_npx f USING(form_id) WHERE f.form_id IS NULL",
    'bridge vote↔cat': "SELECT COUNT(*) cnt FROM proxy_voting_record_category pc LEFT JOIN proxy_voting_record v USING(vote_id) WHERE v.vote_id IS NULL",
    'bridge vote↔mgr': "SELECT COUNT(*) cnt FROM voting_record_manager vm LEFT JOIN proxy_voting_record v USING(vote_id) WHERE v.vote_id IS NULL",
    'bridge vote↔series': "SELECT COUNT(*) cnt FROM voting_record_series vs LEFT JOIN proxy_voting_record v USING(vote_id) WHERE v.vote_id IS NULL"
}
for lbl,sql in checks.items(): qc(lbl, sql, expect_zero=True)

# reverse orphan (defined but never referenced)
qc('unused categories',
   "SELECT COUNT(*) cnt FROM matter_category c LEFT JOIN proxy_voting_record_category pc USING(category_id) WHERE pc.vote_id IS NULL",
   expect_zero=False)

# ── 3. NOT‑NULL sanity ────────────────────────────────────────────────
print('\nNOT‑NULL FIELDS')
qc('form.reporting_person_name NULL',
   "SELECT COUNT(*) cnt FROM form_npx WHERE reporting_person_name IS NULL OR reporting_person_name=''", expect_zero=True)

# ── 4. Business‑logic checks ──────────────────────────────────────────
print('\nBUSINESS‑LOGIC CHECKS (expect 0)')
qc('duplicate accession numbers',
   "SELECT COUNT(*) cnt FROM (SELECT accession_number, COUNT(*) c FROM form_npx GROUP BY accession_number HAVING c>1) x", expect_zero=True)
qc('duplicate series (form,code)',
   "SELECT COUNT(*) cnt FROM (SELECT form_id,series_code,COUNT(*) c FROM series GROUP BY 1,2 HAVING c>1) d", expect_zero=True)
qc('duplicate class (series_id,class_id)',
   "SELECT COUNT(*) cnt FROM (SELECT series_id,class_id,COUNT(*) c FROM series_class GROUP BY 1,2 HAVING c>1) d", expect_zero=True)
qc('negative share fields',
   "SELECT COUNT(*) cnt FROM proxy_voting_record WHERE shares_voted<0 OR shares_on_loan<0 OR vote_cast_shares<0", expect_zero=True)
qc('forms is_parsable=true but zero votes',
   "SELECT COUNT(*) cnt FROM form_npx f LEFT JOIN proxy_voting_record v USING(form_id) WHERE is_parsable AND v.vote_id IS NULL", expect_zero=True)
qc('meeting_date out of range',
   "SELECT COUNT(*) cnt FROM proxy_voting_record WHERE meeting_date>'2100-01-01' OR meeting_date<'1990-01-01'", expect_zero=True)

# series_count check
qc('series_count mismatch',
   "SELECT COUNT(*) cnt FROM form_npx f JOIN (SELECT form_id,COUNT(*) c FROM series GROUP BY form_id) s USING(form_id) WHERE f.series_count<>s.c", expect_zero=True)

# parse success rate (informational)
print('\nPARSE SUCCESS RATE')
rate = q("SELECT ROUND(100.0*SUM(CASE WHEN is_parsable THEN 1 ELSE 0 END)/COUNT(*),1) pct FROM form_npx")[0]['pct']
print(f"Parsed XML for {rate}% of filings\n")

# sample join preview
print('SAMPLE JOIN (5 rows)')
print(pd.DataFrame(q(
    """SELECT v.vote_id, v.issuer_name, m.name manager, c.category_type
           FROM proxy_voting_record v
           LEFT JOIN voting_record_manager vm USING(vote_id)
           LEFT JOIN institutional_manager m USING(manager_id)
           LEFT JOIN proxy_voting_record_category pc USING(vote_id)
           LEFT JOIN matter_category c USING(category_id)
           LIMIT 5""")))

print('\n✅ Data‑quality checks completed successfully.')
